In [31]:
import pandas as pd
import ccxt
import time
import datetime

In [32]:
import json
import requests
import pandas as pd
from tqdm import tqdm
import sqlite3
import datetime


# functions
def get_option_name_and_settlement(coin):
    """ 
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: 2 lists:
                        1.  list of traded options for the selected coin;
                        2.  list of settlement period for the selected coin.
    """

    # requests public API
    r = requests.get("https://test.deribit.com/api/v2/public/get_instruments?currency=" + coin + "&kind=option")
    result = json.loads(r.text)

    # get option name
    name = pd.json_normalize(result['result'])['instrument_name']
    name = list(name)

    # get option settlement period
    settlement_period = pd.json_normalize(result['result'])['settlement_period']
    settlement_period = list(settlement_period)

    return name, settlement_period


def get_option_data(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: pandas data frame with all option data for a given coin
    """

    # get option name and settlement
    coin_name = get_option_name_and_settlement(coin)[0]
    settlement_period = get_option_name_and_settlement(coin)[1]

    # initialize data frame
    coin_df = []

    # initialize progress bar
    pbar = tqdm(total=len(coin_name))

    # loop to download data for each Option Name
    for i in range(len(coin_name)):
        # download option data -- requests and convert json to pandas
        r = requests.get('https://test.deribit.com/api/v2/public/get_order_book?instrument_name=' + coin_name[i])
        result = json.loads(r.text)
        df = pd.json_normalize(result['result'])

        # add settlement period
        df['settlement_period'] = settlement_period[i]

        # append data to data frame
        coin_df.append(df)

        # update progress bar
        pbar.update(1)

    # finalize data frame
    coin_df = pd.concat(coin_df)

    # remove useless columns from coin_df
    columns = ['state', 'estimated_delivery_price']
    coin_df.drop(columns, inplace=True, axis=1)

    # close the progress bar
    pbar.close()

    return coin_df


# print data and time for log
print('Date and time: ' +  datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S") + ' , format: dd/mm/yyyy hh:mm:ss')

# download data -- BTC and ETH Options
btc_data = get_option_data('BTC')

Date and time: 18/10/2023 18:28:23 , format: dd/mm/yyyy hh:mm:ss


  2%|▏         | 9/378 [00:04<03:05,  1.99it/s]

KeyboardInterrupt: 

In [ ]:
r = requests.get("https://test.deribit.com/api/v2/public/get_instruments?currency=" + 'BTC' + "&kind=option").json()
r

{'jsonrpc': '2.0',
 'result': [{'tick_size_steps': [{'above_price': 0.005, 'tick_size': 0.0005}],
   'quote_currency': 'BTC',
   'min_trade_amount': 0.1,
   'expiration_timestamp': 1697702400000,
   'counter_currency': 'USD',
   'settlement_currency': 'BTC',
   'block_trade_tick_size': 0.0001,
   'block_trade_min_trade_amount': 0.1,
   'block_trade_commission': 0.00015,
   'option_type': 'call',
   'settlement_period': 'day',
   'creation_timestamp': 1697443260000,
   'contract_size': 1.0,
   'base_currency': 'BTC',
   'instrument_id': 188135,
   'instrument_type': 'reversed',
   'taker_commission': 0.0003,
   'maker_commission': 0.0003,
   'tick_size': 0.0001,
   'strike': 25500.0,
   'is_active': True,
   'instrument_name': 'BTC-19OCT23-25500-C',
   'kind': 'option',
   'rfq': False,
   'price_index': 'btc_usd'},
  {'tick_size_steps': [{'above_price': 0.005, 'tick_size': 0.0005}],
   'quote_currency': 'BTC',
   'min_trade_amount': 0.1,
   'expiration_timestamp': 1697702400000,
   'co

In [1]:
import requests
import pandas as pd
from datetime import datetime as dt
from datetime import date, timedelta

def datetime_to_timestamp(datetime_obj): 
    """Converts a datetime object to a Unix timestamp in milliseconds."""
    return int(dt.timestamp(datetime_obj)*1000)

def timestamp_to_datetime(timestamp): 
    """Converts a Unix timestamp in milliseconds to a datetime object."""
    return dt.fromtimestamp(timestamp/1000)

def derivative_data(currency: str, kind: str, start_date: date, end_date: date, count: int = 10000) -> pd.DataFrame:
    """Returns derivative trade data for a specified currency and time range.

    Args:
        currency (str): The currency symbol, e.g. 'BTC'.
        kind (str): The type of derivative, either 'option' or 'future'.
        start_date (date): The start date of the time range (inclusive).
        end_date (date): The end date of the time range (inclusive).
        count (int, optional): The maximum number of trades to retrieve per request. Defaults to 10000.

    Returns:
        pandas.DataFrame: A dataframe of derivative trade data for the specified currency and time range.
    """

    # Validate input arguments
    assert isinstance(currency, str), "currency must be a string"
    assert isinstance(start_date, date), "start_date must be a date object"
    assert isinstance(end_date, date), "end_date must be a date object"
    assert start_date <= end_date, "start_date must be before or equal to end_date"

    derivative_list = []
    params = {
        "currency": currency, 
        "kind": kind,
        "count": count,
        "include_old": True,
        "start_timestamp": datetime_to_timestamp(dt.combine(start_date, dt.min.time())),
        "end_timestamp": datetime_to_timestamp(dt.combine(end_date, dt.max.time()))
    }

    url = 'https://history.deribit.com/api/v2/public/get_last_trades_by_currency_and_time'
    with requests.Session() as session:
        while True:
            response = session.get(url, params=params)
            response_data = response.json()
            if len(response_data["result"]["trades"]) == 0:
                break
            derivative_list.extend(response_data["result"]["trades"])
            params["start_timestamp"] = response_data["result"]["trades"][-1]["timestamp"] + 1
            if params["start_timestamp"] >= datetime_to_timestamp(dt.combine(end_date, dt.max.time())):
                break
                
    derivative_data = pd.DataFrame(derivative_list)
    if len(derivative_data) == 0:
        return derivative_data
    derivative_data["date_time"] = pd.to_datetime(derivative_data["timestamp"], unit='ms')
    df_duplicated = derivative_data.duplicated(subset=['trade_id'], keep='first')
    derivative_data = derivative_data[~df_duplicated]
    # derivative_data = derivative_data.drop(['block_trade_id'], axis=1)
    # derivative_data = derivative_data.assign(strike = derivative_data['instrument_name'].str.extract(r'-(\d+.\d+)-'))
    # derivative_data = derivative_data.assign(expire_date = derivative_data['instrument_name'].apply(lambda x : x.split('-')[1]))
    # derivative_data = derivative_data.assign(option_type = derivative_data['instrument_name'].apply(lambda x : x.split('-')[3]))
    # derivative_data['expire_date'] = pd.to_datetime(derivative_data['expire_date'], format='%d%b%y')

    # derivative_data['expiry_date'] =derivative_data['expire_date'] - derivative_data['timestamp']
    # derivative_data = derivative_data.sort_values(by=['date_time', 'strike', 'expiry_date', 'option_type'], ascending=[True, True, True, True])


    return derivative_data

data = derivative_data('BTC', 'option', date(2023, 1, 1), date(2023, 2, 2), 100)
data.to_csv('output.csv')

KeyError: "['block_trade_id'] not found in axis"

In [61]:
data['instrument_name'].str

In [76]:
df_duplicated = data.duplicated(subset=['trade_id'], keep='first')
df_unique = data[~df_duplicated]
df_unique = df_unique.drop(['block_trade_id'], axis=1)
df_unique['timestamp'] = pd.to_datetime((df_unique['timestamp']), unit='ms')
df_unique = df_unique.assign(strike = df_unique['instrument_name'].str.extract(r'-(\d+.\d+)-'))
df_unique = df_unique.assign(expire_date = df_unique['instrument_name'].apply(lambda x : x.split('-')[1]))
df_unique = df_unique.assign(option_type = df_unique['instrument_name'].apply(lambda x : x.split('-')[3]))

df_unique['expire_date'] = pd.to_datetime(df_unique['expire_date'], format='%d%b%y')

# Calculate 'expiry_date' by subtracting 'expire_date' from 'timestamp'
df_unique['expiry_date'] =df_unique['expire_date'] - df_unique['timestamp']
df_unique = df_unique.sort_values(by=['date_time', 'strike', 'expiry_date', 'option_type'], ascending=[True, True, True, True])
# # Convert 'expiry_date' column to date format
# df_unique['expiry_date'] = df_unique['expiry_date'].apply(lambda x: x.date())

df_unique.to_csv('output.csv')

In [87]:
dt.strptime(str(df_unique['expire_date'][0]), '%Y-%m-%d %H:%M:%S')


datetime.datetime(2022, 2, 18, 0, 0)

In [27]:
# r = requests.get('https://test.deribit.com/api/v2/public/get_instruments?currency=BTC&kind=option').json()
r = requests.get('https://test.deribit.com/api/v2/public/get_order_book_by_instrument_id?instrument_id=187151&depth=1').json()
# r = requests.get('https://test.deribit.com/api/v2/public/get_index_price?index_name=btc_usd').json()
r

{'jsonrpc': '2.0',
 'error': {'message': 'not_found',
  'data': {'key': 187151, 'object': 'instrument_id'},
  'code': 13020},
 'testnet': True,
 'usIn': 1697676738812682,
 'usOut': 1697676738812750,
 'usDiff': 68}